# iLOrest examples


Version 0.57

## Introduction

This Jupyter Notebook contains iLOrest [examples](#BackToList) as well as tips and tricks answering recurrent questions from HPE customers.

## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.25) ############


# Set Student ID number
Stud={{ STDID }}
Id=$(id --user --name)

# location and ports variables
ObmcBasePort=44000
IloDlBasePort=45000
IloSyBasePort=46000
let OpenBmcBasePort=$ObmcBasePort
let OpenBmcPort=${OpenBmcBasePort}+${Stud}
let iLO5SimulatorBasePort=$IloDlBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${Stud}
let ilo5Port=443

CacheDir="${PWD}/iLOrestCache"
LogDir="${PWD}"
LogFile="${LogDir}/iLOrest.log"

#iLO5SimulatorIP=bmcsimulators
iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

OpenBmcIP=openbmcsimulators
OpenBmc="${OpenBmcIP}:${OpenBmcPort}"
OpenBmcURI="https://${OpenBmc}"

ilo5IP="ilo5"
ilo5="${ilo5IP}:${ilo5Port}"
ilo5URI="https://${ilo5}"

# Credentials
User="student"
Password='P@ssw0rd!'

# Miscellaneous
alias ilorest='ilorest --nologo --cache-dir ${CacheDir} --logdir ${LogDir}'
alias ResetIlo5Simulator="ssh rstsimul@${iLO5SimulatorIP} $Id $iLO5SimulatorBasePort"
ilorest logout &>/dev/null # Clear cache

alias ResetIlo5Simulator

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

### iLO 5 login and cache redirection

This [infrastructure](Pictures/ProgrammingRedfishInfraDescription.png) contains only one physical iLO 5 and prevents multiple students to perform SET actions on this unique BMC. Therefore, students have to use a DMTF BMC simulator populated with an iLO 5 mockup.

However, because `iLOrest` populates a cache directory during the login process, you need first to authenticate with the iLO 5 and then modify the cache files to have them pointing to the DMTF simulator. This is the role of the next cell.

In [ ]:
ilorest login $ilo5 -u $User -p $Password

# Copy cache in a safe location before logging out to
# minimize opened sessions. iLO 5 is limited to 15 concurrent
# sessions. 
mv ${CacheDir}/cache{,-bck}
ilorest logout

# Move back the cache
mv ${CacheDir}/cache{-bck,}

# Redirect cache toward the iLO 5 Simulator
sed -i "s?${ilo5URI}?${iLO5SimulatorURI}?" ${CacheDir}/cache/*

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart you iLO 5 simulator to start this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart
ResetIlo5Simulator

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## <a id = "BackToList">List of examples</a>

Note that the list of examples present in this Jupyter Notebook is changing over time. The classification is not following any strict model; it is a mix of Root service and data type locations.


<table>
    <tr>
        <th> &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; </th>
        <th> &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; </th>
    </tr>
    <tr>
        <td>    
            
Account Service
   - [iLO users: Enforce password complexity and minimum password length](#EnforceiLOUsersPasswordComplexity)
        
        </td>
        <td>
            
Chassis
   - [Chassis Serial Number](#ChassisSerialNumber)
   - [Power State](#PowerState)
   - [Indicator LED](#IndicatorLED)
   - [Chassis Asset Tag](#ChassisAssetTag)
            
        </td>
    </tr>
    <tr>
        <td>
            
Thermal
   - [Server thermal information](#ThermalInfo)
   - [Thermal configuration](#ThermalConfiguration)
            
        </td>
        <td>
            
Managers (iLO)
   - [Enable/Disable SNMPv1 in iLO](#SNMPv1Enabled)

        </td>
    </tr>
    <tr>
        <td>
            
CompusterSystem
   - [PostState and DiscoveryComplete](#PostState_And_DiscoveryComplete)
   - [AMS and HostOS](#AMS_And_HostOS)
   - [Bios/RBSU Login password](#ManageBiosRbsuPassword)
   - [Computer System Asset Tag](#ComputerSystemAssetTag)
                       
        </td>
        <td>
            
Resource Directory
   - [Introduction](#ResourceDirectoryIntroduction)             
             
        </td>
    </tr>
    <tr>
        <td>
            
Security
   - [Security Dashboard](#SecurityDashboard)
   - [iLO Security Mode/State](#iLOSecurityMode)
   - [Login Security Banner](#LoginSecurityBanner)

        </td>
        <td>
            
Bios
   - [Server Serial Number](#BiosSerialNumber)
                      
        </td>       
    </tr>
    <tr>
        <td>
            
Storage
   - [SSD Wear Status](#SSD_WearStatus)            
            
        </td>
        <td>
            
Logout
   - [Logout](#Logout)            
            
        </td>
    </tr>
</table>

## Account Service

This root service models the iLO User Accounts.

### <a id = "EnforceiLOUsersPasswordComplexity">iLO users: Enforce password complexity and minimum password length</a>

API Reference document keyword: `EnforcePasswordComplexity` \
iLO GUI location: `Security - Access Settings`

In [ ]:
# Select DMTF data type 
ilorest select AccountService

# List properties of the HPE Oem extension in json format 
ilorest list Oem/Hpe --json

# Modify two AccountService parameters
ilorest set Oem/Hpe/EnforcePasswordComplexity=True
ilorest set Oem/Hpe/MinPasswordLength=5

# Retrieve the list of modified paramters
ilorest status

# Uncommnent the following comand to commit changes in the iLO 5 simulator.
# Note that you will not get the same answer as if done on a real iLO 5.
#ilorest commit

# Undo changes (clear cache)
ilorest select AccountService --refresh

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Chassis

This section contains parameters related to the `Chassis` [data type](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#chassis-v1_6_0-chassis). A Chassis represents a physical or virtual container of compute resources with attrbutes such as FRU information, power supplies, temperature, etc.

Complex chassis subsystems may have their own data type (i.e. [Thermal](#ThermalInfo))

Chassis examples:
  - Rack
  - Blade enclosure
  - HPE Synergy frame 
  - Compute node chassis

### <a id = "ChassisSerialNumber">Chassis Serial Number</a>

The chassis serial number of an HPE compute node mentioned on an external label/tag with code bar/QRcode is identical to its embedded server serial number (Bios parameter). Note that the serial number of a C7000 enclosure chassis or Synergy frame chassis is unique and different to every SN of embedded compute nodes. 

The following cell retrieves the serial number of an HPE compute node chassis and shows explicitely that it is a `read-only` property. The server serial number is presented [here](#BiosSerialNumber).

[API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#serialnumber294) keyword: `SerialNumber` \
iLO GUI location: `Information - iLO Overview` 



In [ ]:
# Select data type
ilorest select Chassis

# Get SerialNumber from the Chassis data type
ilorest get SerialNumber

# Note that this property is READ-ONLY
ilorest info SerialNumber

### <a id = "PowerState">Power State</a>

The following cell retrieves the power state and information related to this parameter. Note that this parameter is read-only. If you want a more precise server state, read the [PostState and DiscoveryComplete](#PostState_And_DiscoveryComplete) and [AMS and HostOS](#AMS_And_HostOS) paragraphs.

As mentioned in the [API reference documentation](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#powerstate) this property is part of the `Chassis` data type. 

In [ ]:
# Select data type
ilorest select Chassis

# Get property value
ilorest get PowerState

# Retrieve property info with possible values.
ilorest info PowerState



### <a id = "IndicatorLED">Indicator LED</a>

The chassis indicator LED corresponds to the blue LED that is used to identify the chassis in a rack of the datacenter.

In [ ]:
# Select data type
ilorest select Chassis

# Get property value
IndicatorLED=$(ilorest get --json IndicatorLED | jq -r '.IndicatorLED')
echo "IndicatorLED=$IndicatorLED"

# Retrieve property info
ilorest info IndicatorLED

In [ ]:
# Refresh cache
ilorest select Chassis --refresh

# Change the indicator LED state
LEDState=$(ilorest get IndicatorLED | cut -d= --field=2)
echo -e "LED State:$LEDState\n"  

if [ "$LEDState" = "Off" ]; then
    LEDState="Lit"
else
    LEDState="Off"
fi

# Change property and commit
echo "Changing IndicatorLED to $LEDState"
ilorest set IndicatorLED="$LEDState" --commit
echo

# Validate property change
ilorest get IndicatorLED

### <a id = "ChassisAssetTag">Chassis Asset Tag</a>

The Chassis `AssetTag` is different from the `ComputerSystem` [Asset Tag](#ComputerSystemAssetTag). 

In [ ]:
# Select data type
ilorest select Chassis --refresh

# Get property value
ilorest get AssetTag

In [ ]:
# Change the Chassis Asset Tag property
ilorest set AssetTag="New Chassis Asset Tag" --commit

# Verify change
ilorest get AssetTag

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Thermal

The [Thermal](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#thermal-v1_1_0-thermal) data type describes the system thermal metrics. It provides the temperatures in different locations in the chassis as well as fan properties. It also offers the possibility to configure the fans with a specific cooling directive (Optimal, Enhanced, etc ...). 

Thermal information belongs to the `Thermal` data type. However, the thermal configuration (`ThermalConfig`) is part of the `Bios` data type. Hence, to modify the `ThermalConfig` property, you will need to reboot the entire system to effectively apply this modification.

However, starting with iLO firmware **2.30**, it is possible to modify the thermal configuration using the `ThermalConfiguration` property of the `Thermal` data type without any system reset. 

>**NOTE**: The `Thermal` schema implemented in this iLO firmware version (2.30) is not totally accurate and only an `ilorest rawpatch` command can modify this setting. This problem will be fixed in a later update.

### <a id = "ThermalInfo">Server Thermal Information</a>

The following cell retrieves both fan and sensor properties, including temperatures.

In [ ]:
# Retrieve server thermal properties
ilorest list --json --select Thermal

If you want only sensor information, you can use the **`serverinfo`** macro command like that found in the following cell

In [ ]:
# Retrieve data type used by the server info macro command
ilorest serverinfo --thermal

### <a id = "ThermalConfiguration">ThermalConfiguration</a>

The following cells retrieve the thermal configuration from the `Bios` data type first, and then from the `Thermal` data type.

In [ ]:
# Select data type
ilorest select Bios

# Retrieve property
ilorest get Attributes/ThermalConfig --json | jq '.[]'

# Retrieve possible values for the ThermalConfig property
#TBD

# Change ThermalConfig
#ilorest set ThermalConfig="EnhancedCPUCooling" --select Bios.

In [ ]:
# Verify iLO firmware is equal or greater than 2.30
ilorest get FirmwareVersion --select Manager.

# Select data type
ilorest select Thermal

# Retrieve property
ilorest get Oem/Hpe/ThermalConfiguration --json | jq -r '.[]|.[]'

# Retrieve possible values for property
# TBD

# Modify property
#ilorest set Oem/Hpe/ThermalConfiguration="MaximumCooling"



<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Managers

Examples in this section are related to the `Manager` data type as well as HPE Oem data types located below the `/redfish/v1/Managers/` end point like the `HpeiLOSnmpService`. 

### <a id = "SNMPv1Enabled">Enable/Disable SNMPv1 in iLO</a>

> **NOTE** Changing the `SNMPv1Enabled` setting appeared in iLO 5 FW **2.30**.

    API Reference document keyword: "SNMPv1Enabled" \
    iLO GUI location: Management --> SNMP Settings --> SNMP Alerts --> SNMPv1

In [ ]:
# Select data type
ilorest select HpeiLOSnmpService

# Get current value
ilorest get SNMPv1Enabled
echo

# Set new value
ilorest set SNMPv1Enabled=False

ilorest status

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## HPE iLO Resource Directory

The HPE iLO Resource Directory is an HPE OEM extention that contains the exhaustive list of the data types present in a Redfish service. This directory can be used by Redfish clients to lookup resources faster. Without this extension, Redfish clients must crawl the entire Redfish tree, starting at the root entry point, to find a specific resource.

The iLOrest utility caches the HPE iLO Resource Directory locally during the login process. When you select a data type, iLOrest very quickly retrieves its location from the cached directory. Then, when you submit get or set requests against specific properties, iLOrest starts looking for them starting at the location found in the cached directory.


### <a id = "ResourceDirectoryIntroduction">Introduction</a>

The `HpeiLOResourceDirectory` data type contains an array called `Instances`. Each element of this array corresponds to a data type with associated descriptors: `@odata.id` and `HttpMethods`. The first descriptor is the entry point of the data type in the Redfish tree and the second lists the valid HTTP methods against the data type.

> NOTE: The word "instance" is a synonym of "data type". You will find it in DMTF [python programs](https://github.com/DMTF/Redfish-Tacklebox/blob/master/redfish_utilities/sensors.py) as well as [HPE programs](https://github.com/HewlettPackard/python-ilorest-library/blob/master/examples/Redfish/get_resource_directory.py).

The following cell retrieves the first data types (aka instances) of the current iLO 5.

In [ ]:
# Select data type
ilorest select HpeiLOResourceDirectory

# List the first elements of array Instances containing data type properties
ilorest list Instances --json | head -30

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Computer System

This section contains examples related to the `ComputerSystem` data type.

### <a id = "PostState_And_DiscoveryComplete">PostState and DiscoveryComplete</a>

Use these state parameters to evaluate the current state of a system before modifying or viewing its resources. As explained in this [article](https://developer.hpe.com/blog/master-the-redfish-server-states-to-improve-your-monitoring-and-manageme), certain parameters cannot be modified if the computer is not in a specific state.

Moreover, some parameters cannot be viewed when the entire discovery of all subsystems is not complete. As an example, you will not be able to view or modify SmartArray and logical drive configurations if the `SmartArrayDiscovery` is not complete.

In [ ]:
# Select data type
ilorest select ComputerSystem

# Retrieve and print DeviceDiscoveryComplete and PostState properties
ilorest get Oem/Hpe/DeviceDiscoveryComplete | grep -v -E 'Oem|Hpe'; echo
ilorest get Oem/Hpe/PostState | grep -v -E 'Oem|Hpe' ; echo

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "AMS_And_HostOS">AMS and HostOS</a>

When the HPE Agentless Management Service (AMS) is installed in the operating system of the host, it is possible to get various information from the `HostOS` property. This property is present only when AMS is up and running in the host OS.

Use keywords `agentless` or `ams` in the [API reference documentation](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#introduction) to get the data type related to this topic.

In [ ]:
# Select type
ilorest select ComputerSystem.

# Retrieve and print AMS status
AmsStatus=$(ilorest get --json Oem/Hpe/AggregateHealthStatus/AgentlessManagementService | \
jq -r '.Oem.Hpe.AggregateHealthStatus.AgentlessManagementService')
echo "AMS Status: $AmsStatus" ; echo

# Retrieve and print HostOS info only if AMS is ready
if [ "$AmsStatus" == "Ready" ] ; then
    echo "Host OS properties:"
    ilorest get --json Oem/Hpe/HostOS | jq -r '.Oem.Hpe.HostOS'
fi

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "ManageBiosRbsuPassword">Bios/RBSU Login password</a>

In [ ]:
# Setting a password for the first time
ilorest setpassword FirstTimePassword '""'

# Removing the Bios/RBSU password
ilorest setpassword '""' FirstTimePassowrd

# No commit needed. No iLO nor system reset needed. 
ilorest status


<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "ComputerSystemAssetTag">ComputerSystem Asset Tag</a>

The `ComputerSystem` Asset Tag is different from the `Chassis` [Asset Tag](#ChassisAssetTag).

In [ ]:
# Select data type and clear cache
ilorest select ComputerSystem. --refresh

# Retrieve ComputerSystem Asset Tag
ilorest get AssetTag

# Change ComputerSystem Asset Tag
ilorest set AssetTag="New ComputerSystem Asset Tag" --commit

# Verify change
ilorest get AssetTag

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Security

### <a id = "SecurityDashboard">Security Dashboard</a>

The following five examples retrieve different security dashboard properties.

#### Overall security dashboard status

Use keyword `Security Dashboard` or `SecurityDashboard` in the [API reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#hpeilosecuritydashboard-v1_0_0-hpeilosecuritydashboard) to find the overall security dashboard data type.

In [ ]:
# Select data type
ilorest select HpeiLOSecurityDashboard.

# Retrieve overall security dashboard status 
ilorest get 


#### All Security parameters and properties in json format

Use keyword `SecurityParam` to find the corresponding data type in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#hpeilosecurityparam-v1_1_0-hpeilosecurityparam).

In [ ]:
# Select data type
ilorest select HpeiLOSecurityParam.

# List security params in JSON format
ilorest list --json

#### List all security params at risk

To retrieve this list you don't need to select the data type as it is already selected.

In [ ]:
ilorest list --filter SecurityStatus="Risk"

#### Get specific security parameter properties

In [ ]:
ilorest get --filter Name="Require Login for iLO RBSU"

#### Get specific property of a specific security parameter

In [ ]:
ilorest get Ignore --filter Name="Minimum Password Length"

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "iLOSecurityMode">Security Mode/State</a>

From the iLO 5 Graphical User Interface, in the `Security` --> `Encryption` menu, you can view and modify the `SecurityState` setting. This setting is part of the `HpeSecurityService` data type as mentioned in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#securitystate).

>NOTE: Software like the Smart Update Tool (`SUT`) may refer to this setting as `Security Mode` instead of `Security State`. 

#### Retrieve and modify the iLO 5 Security State setting

The following cell retrieves all the properties of the `HpeSecurityService` and modifies the `SecurityState`. A similar change on a real iLO 5 **automatically triggers a reset** of the iLO. 

In [ ]:
# Select data type and refresh cache
ilorest select HpeSecurityService. --refresh

# Retrieve all Security Service paramters
ilorest get --json

# Modify Security State
SecurityState=$(ilorest get SecurityState | cut --delimiter== --field=2)
echo 

if [ "$SecurityState" = "Production" ]; then
   echo "Setting Security State to HighSecurity"
   ilorest set SecurityState="HighSecurity" --commit
else
    echo "Setting Security State to Production"
    ilorest set SecurityState="Production" --commit
fi

echo
ilorest get SecurityState 

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "LoginSecurityBanner">Login Security Banner</a>

From the iLO 5 Graphical User Interface, in the `Security` --> `Login Security Banner` menu, you can enable/disable and customize the `LoginSecurityBanner` setting. This setting is part of the `HpeSecurityService` data type as mentioned in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#loginsecuritybanner).

#### Retrieve and modify the iLO 5 Security State setting

The following cell modifies the state of the Security Login Banner and provides a message when enabled. You don't need to reset the iLO 5 to effectively modify this banner.

In [ ]:
# Select data type and refresh cache
ilorest select HpeSecurityService. --refresh

# Retrieve Login Security Banner settings
ilorest get LoginSecurityBanner --json

# Modify Login Security Banner settings
LoginSecurityBanner=$(ilorest get LoginSecurityBanner/IsEnabled | cut --delimiter== --field=2)
echo 

if [ $LoginSecurityBanner = False ]; then
   echo "Enabling Login Security Banner with a custom message"
   ilorest set LoginSecurityBanner/IsEnabled=True --commit
   ilorest set LoginSecurityBanner/SecurityMessage="This is My Custom Security Message" --commit
else
    echo "Disabling Login Security Banner"
    # The following set command is not required with a real iLO 5. 
    # It's only there to get a "realistic" behavior of the simulator.
    ilorest set LoginSecurityBanner/SecurityMessage="" --commit
    ilorest set LoginSecurityBanner/IsEnabled=False --commit
fi

echo
ilorest get --json LoginSecurityBanner 

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Bios

### <a id = "BiosSerialNumber">Server Serial Number stored in BIOS</a>

In a computer node, the Serial Number stored in the `Bios.` data type corresponds to the server mother board serial number. HPE requires the change of this serial number in case of mother board replacement to match the chassis serial number. 

In [ ]:
# Select data type
ilorest select Bios. --refresh
echo

# Make sure SN is read-write
ilorest info Attributes/SerialNumber --latestschema
echo

# Get SerialNumber from Bios data type.
ilorest get Attributes/SerialNumber

echo "Set new SN"
ilorest set Attributes/SerialNumber="NEWSN" --commit

# Reload a fresh simulator cache
ilorest select Bios. --refresh
echo 

# Validate the SN change
ilorest get Attributes/SerialNumber


## Storage

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "SSD_WearStatus">SSD Wear Status</a>

The `WearStatus` property belongs to the `Drive` data type as mentioned in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#oem-hpe-wearstatus).

The `Drive` data type may not exist in servers not containing suitable hardware. Unfortunately, the physical DL360 Gen10 server available in this infrastrucutre (and associated simulator) does not contain any SSD drive with a `WearStatus`. Hence, you will have to switch to another iLO 5 simulator.

The next cell points to another iLO 5 simulator containing suitable hardware.


In [ ]:
# Switch iLO 5 Simulator
PreviousiLO5SimulatorURI=${iLO5SimulatorURI}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${Stud}
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}
alias ResetIlo5Simulator="ssh rstsimul@${iLO5SimulatorIP} $Id $iLO5SimulatorBasePort"

# iLO 5 Simulator restart
ResetIlo5Simulator

# Verify all BMCs are reachable
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Change cache with Synergy simulator content and SSD drives
rm ${CacheDir}/cache/* &>/dev/null
tar -C ${CacheDir}/cache -x -f SSD-cache.tgz 


# Redirect cache toward the iLO 5 Simulator
sed -i "s?${iLO5SimulatorBasePort}?${iLO5SimulatorPort}?" ${CacheDir}/cache/*

echo ; echo
echo "NOTE: This notebooks points now to a Synergy iLO 5 simulator"
echo "      If you want to come back to the DL360g10 iLO 5 simulator,"
echo "      restart Kernel and clear all output and run the first two cells (environment preparation and login)
echo "      at the beginning of this Notebook"
echo ; echo

   &nbsp;


The Redfish service of this new simulated compute node contains several SSD disk drives. Among them, only two of them have the `WearStatus` propoerty. The next cell identifies these two drives and then retrieves the `WearStatus` of one of them.

In [ ]:
echo "Drives in the Chassis data type"
ilorest list Links/Drives --json --select Chassis.

echo -e "\n Wear status of drive 3d9db7a1" 
ilorest get Oem/Hpe/WearStatus --json --filter Id="3d9db7a1" --select Drive

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## <a id = "Logout">Logout</a>

Always logout when your session is over. You can as well logout and [restart](#iLO5SimulatorRestart) the iLO 5 simulator if you want once again use this Jupyter Notebook.

In [ ]:
ilorest logout

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Summary

In this workshop, you discovered several HPE iLOrest examples. Go to the [Conclusion Notebook](3-Conclusion.ipynb) if you are finished.